stableline3 与 gym 是辅助使用的

gym 提供环境部分，也很方便创建环境，考虑自定义部分
stableline3 提供丰富的算法部分

学习主要参照教程：
https://blog.csdn.net/tianjuewudi/article/details/123113885

## 种类1：gym + pytorch

说明：封装整理工作量极大，编程复用性不高
推广各种算法部分，使用 stableline3 就产生了

In [3]:
import gym
from gym import wrappers

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np

from IPython.display import clear_output
from matplotlib import pyplot as plt

import random
from timeit import default_timer as timer
from datetime import timedelta
import math
from utils.wrappers import make_atari, wrap_deepmind, wrap_pytorch

from utils.hyperparameters import Config
from agents.BaseAgent import BaseAgent


config = Config()

config.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#epsilon variables
config.epsilon_start = 1.0
config.epsilon_final = 0.01
config.epsilon_decay = 30000
config.epsilon_by_frame = lambda frame_idx: config.epsilon_final + (config.epsilon_start - config.epsilon_final) * math.exp(-1. * frame_idx / config.epsilon_decay)

#misc agent variables
config.GAMMA=0.99
config.LR=1e-4

#memory
config.TARGET_NET_UPDATE_FREQ = 1000
config.EXP_REPLAY_SIZE = 100000
config.BATCH_SIZE = 32

#Learning control variables
config.LEARN_START = 10000
config.MAX_FRAMES=1000000

class ExperienceReplayMemory:
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []

    def push(self, transition):
        self.memory.append(transition)
        if len(self.memory) > self.capacity:
            del self.memory[0]

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)


class DQN(nn.Module):
    def __init__(self, input_shape, num_actions):
        super(DQN, self).__init__()

        self.input_shape = input_shape
        self.num_actions = num_actions

        self.conv1 = nn.Conv2d(self.input_shape[0], 32, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)

        self.fc1 = nn.Linear(self.feature_size(), 512)
        self.fc2 = nn.Linear(512, self.num_actions)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x

    def feature_size(self):
        return self.conv3(self.conv2(self.conv1(torch.zeros(1, *self.input_shape)))).view(1, -1).size(1)


class Model(BaseAgent):
    def __init__(self, static_policy=False, env=None, config=None):
        super(Model, self).__init__()
        self.device = config.device

        self.gamma = config.GAMMA
        self.lr = config.LR
        self.target_net_update_freq = config.TARGET_NET_UPDATE_FREQ
        self.experience_replay_size = config.EXP_REPLAY_SIZE
        self.batch_size = config.BATCH_SIZE
        self.learn_start = config.LEARN_START

        self.static_policy = static_policy
        self.num_feats = env.observation_space.shape
        self.num_actions = env.action_space.n
        self.env = env

        self.declare_networks()

        self.target_model.load_state_dict(self.model.state_dict())
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)

        # move to correct device
        self.model = self.model.to(self.device)
        self.target_model.to(self.device)

        if self.static_policy:
            self.model.eval()
            self.target_model.eval()
        else:
            self.model.train()
            self.target_model.train()

        self.update_count = 0

        self.declare_memory()

    def declare_networks(self):
        self.model = DQN(self.num_feats, self.num_actions)
        self.target_model = DQN(self.num_feats, self.num_actions)

    def declare_memory(self):
        self.memory = ExperienceReplayMemory(self.experience_replay_size)

    def append_to_replay(self, s, a, r, s_):
        self.memory.push((s, a, r, s_))

    def prep_minibatch(self):
        # random transition batch is taken from experience replay memory
        transitions = self.memory.sample(self.batch_size)

        batch_state, batch_action, batch_reward, batch_next_state = zip(*transitions)

        shape = (-1,) + self.num_feats

        # (32,1,84,84)
        batch_state = torch.tensor(batch_state, device=self.device, dtype=torch.float).view(shape)
        # (32,1)
        batch_action = torch.tensor(batch_action, device=self.device, dtype=torch.long).squeeze().view(-1, 1)
        # (32,1)
        batch_reward = torch.tensor(batch_reward, device=self.device, dtype=torch.float).squeeze().view(-1, 1)
        # map()会根据提供的函数对指定序列做映射，这里检查下一个状态是否为空，shape为(32)
        non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, batch_next_state)), device=self.device,
                                      dtype=torch.uint8)
        try:  # sometimes all next states are false
            # 检查状态中的数是否存在，shape为(32,1,84,84)
            non_final_next_states = torch.tensor([s for s in batch_next_state if s is not None], device=self.device,
                                                 dtype=torch.float).view(shape)
            empty_next_state_values = False
        except:
            non_final_next_states = None
            empty_next_state_values = True

        return batch_state, batch_action, batch_reward, non_final_next_states, non_final_mask, empty_next_state_values

    def compute_loss(self, batch_vars):
        # 提取六组数据
        batch_state, batch_action, batch_reward, non_final_next_states, non_final_mask, empty_next_state_values = batch_vars

        # estimate
        current_q_values = self.model(batch_state).gather(1, batch_action)

        # target
        with torch.no_grad():
            max_next_q_values = torch.zeros(self.batch_size, device=self.device, dtype=torch.float).unsqueeze(dim=1)
            if not empty_next_state_values:
                max_next_action = self.get_max_next_state_action(non_final_next_states)
                max_next_q_values[non_final_mask] = self.target_model(non_final_next_states).gather(1, max_next_action)
            expected_q_values = batch_reward + (self.gamma * max_next_q_values)

        diff = (expected_q_values - current_q_values)
        loss = self.huber(diff)
        loss = loss.mean()

        return loss

    def update(self, s, a, r, s_, frame=0):
        if self.static_policy:
            return None

        self.append_to_replay(s, a, r, s_)

        if frame < self.learn_start:
            return None

        batch_vars = self.prep_minibatch()

        loss = self.compute_loss(batch_vars)

        # Optimize the model
        self.optimizer.zero_grad()
        loss.backward()
        for param in self.model.parameters():
            param.grad.data.clamp_(-1, 1)
        self.optimizer.step()

        self.update_target_model()
        self.save_loss(loss.item())
        self.save_sigma_param_magnitudes()

    def get_action(self, s, eps=0.1):
        with torch.no_grad():
            if np.random.random() >= eps or self.static_policy:
                X = torch.tensor([s], device=self.device, dtype=torch.float)
                a = self.model(X).max(1)[1].view(1, 1)
                return a.item()
            else:
                return np.random.randint(0, self.num_actions)

    def update_target_model(self):
        self.update_count += 1
        self.update_count = self.update_count % self.target_net_update_freq
        if self.update_count == 0:
            self.target_model.load_state_dict(self.model.state_dict())

    def get_max_next_state_action(self, next_states):
        return self.target_model(next_states).max(dim=1)[1].view(-1, 1)

    def huber(self, x):
        cond = (x.abs() < 1.0).to(torch.float)
        return 0.5 * x.pow(2) * cond + (x.abs() - 0.5) * (1 - cond)


def plot(frame_idx, rewards, losses, sigma, elapsed_time):
    clear_output(True)
    plt.figure(figsize=(20,5))
    plt.subplot(131)
    plt.title('frame %s. reward: %s. time: %s' % (frame_idx, np.mean(rewards[-10:]), elapsed_time))
    plt.plot(rewards)
    if losses:
        plt.subplot(132)
        plt.title('loss')
        plt.plot(losses)
    if sigma:
        plt.subplot(133)
        plt.title('noisy param magnitude')
        plt.plot(sigma)
    plt.show()


start = timer()

env_id = "PongNoFrameskip-v4"
env = make_atari(env_id)
env = wrap_deepmind(env, frame_stack=False)
env = wrap_pytorch(env)
model = Model(env=env, config=config)

episode_reward = 0

observation = env.reset()
for frame_idx in range(1, config.MAX_FRAMES + 1):
    epsilon = config.epsilon_by_frame(frame_idx)

    action = model.get_action(observation, epsilon)
    prev_observation = observation
    observation, reward, done, _ = env.step(action)
    observation = None if done else observation

    # 如果没有更新的动作就是表示已经再测试使用了
    model.update(prev_observation, action, reward, observation, frame_idx)
    episode_reward += reward

    if done:
        print("Finish a episode")
        observation = env.reset()
        model.save_reward(episode_reward)
        episode_reward = 0

        if np.mean(model.rewards[-10:]) > 19:
            print("mean_reward>19....Plot...")
            plot(frame_idx, model.rewards, model.losses, model.sigma_parameter_mag,
                 timedelta(seconds=int(timer() - start)))
            break

    if frame_idx % 10000 == 0:
        plot(frame_idx, model.rewards, model.losses, model.sigma_parameter_mag, timedelta(seconds=int(timer() - start)))
        print("Plot..................")

model.save_w()
env.close()


ModuleNotFoundError: No module named 'utils.wrappers'

### 官方的 e.g

In [10]:
# # 没有经过训练

# import gymnasium as gym

# from stable_baselines3 import A2C
# from stable_baselines3 import PPO

# env = gym.make("CartPole-v1", render_mode="human")

# vec_env = model.get_env()
# obs = vec_env.reset()
# for i in range(1000):
#     action, _state = model.predict(obs, deterministic=True)
#     obs, reward, done, info = vec_env.step(action)
#     vec_env.render("human")
#     # VecEnv resets automatically
#     # if done:
#     #   obs = vec_env.reset()

TypeError: __init__() got an unexpected keyword argument 'render' was raised from the environment creator for CartPole-v1 with kwargs ({'render': 'human'})

In [1]:
import gymnasium as gym

from stable_baselines3 import A2C
from stable_baselines3 import PPO


env = gym.make("CartPole-v1", render_mode="human")

# model = A2C("MlpPolicy", env, verbose=1)
model = A2C(
    "MlpPolicy",
    env=env,
    learning_rate=1e-3,

    verbose=1,
    tensorboard_log="./tensorboard/CartPole-v0/" # 可以考虑替换为 wandb
)

model.learn(total_timesteps=10_000)

vec_env = model.get_env()
obs = vec_env.reset()
for i in range(1000):
    action, _state = model.predict(obs, deterministic=True)
    obs, reward, done, info = vec_env.step(action)
    vec_env.render("human")
    # VecEnv resets automatically
    # if done:
    #   obs = vec_env.reset()

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./tensorboard/CartPole-v0/A2C_3
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 44.2     |
|    ep_rew_mean        | 44.2     |
| time/                 |          |
|    fps                | 37       |
|    iterations         | 100      |
|    time_elapsed       | 13       |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -0.336   |
|    explained_variance | 0.617    |
|    learning_rate      | 0.001    |
|    n_updates          | 99       |
|    policy_loss        | 1.98     |
|    value_loss         | 5.72     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 42.4     |
|    ep_rew_mean        | 42.4     |
| time/                 |          |
|    fps                | 41       |
|    iterat

## stable_baseline3 快速使用

In [ ]:
import tensorboard as tb

tensorboard 使用

^C


In [1]:
from stable_baselines3 import DQN
# 定义其中的环境部分
from stable_baselines3.common.vec_env.dummy_vec_env import DummyVecEnv
# 导入策略评估
from stable_baselines3.common.evaluation import evaluate_policy
import gym

env_name ="CartPole-v1"
env = gym.make(env_name) # 注意环境可以创建多个，最后只要是向量式的相关返回即可

env = DummyVecEnv([lambda: env]) # 将环境转换为向量式的环境

# 定义其中的DQN模型，设置参数部分
model = DQN(
    "MlpPolicy",
    env=env,
    learning_rate=1e-3,
    batch_size=64,
    buffer_size=100000,
    learning_starts=0,
    target_update_interval=500,
    policy_kwargs={"net_arch":[256,256]}, # 代表里面创建的相关网络结构
    verbose=1,
    tensorboard_log="./tensorboard/CartPole-v0/" # 可以考虑替换为 wandb
)

# 开始训练
model.learn(total_timesteps=1e5)
# TODO,下面还会单独运行吗

# 策略评估，装载开始训练
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10, render=True)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

# 保存模型参数
model.save("dqn_c./model/CartPole.pkl")

c:\Users\peter\.conda\envs\torch_env\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cuda device
Logging to ./tensorboard/CartPole-v0/DQN_1


c:\Users\peter\.conda\envs\torch_env\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


----------------------------------
| rollout/            |          |
|    exploration_rate | 0.993    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 182      |
|    time_elapsed     | 0        |
|    total_timesteps  | 73       |
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 0.00844  |
|    n_updates        | 18       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.981    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 415      |
|    time_elapsed     | 0        |
|    total_timesteps  | 195      |
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 0.000591 |
|    n_updates        | 48       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rat

KeyboardInterrupt: 

In [ ]:
!tensorboard --logdir .\tensorboard\CartPole-v0\

## 读取已有的模型来控制环境

## 各种参数的参数替换

https://stable-baselines3.readthedocs.io/en/master/modules/base.html

## 高级的保存、加载

因为 stable_baseline3 里面虽然方便替换了各种算法
但是里面的很多详细参数很复杂，这里需要重新进行考虑操作

## 注册环境具体如何进行操作

## 如何在 stable_baseline3 上基本进行改变操作（比如个性化定制某部分）

# 补充说明

- 类似的算法库有很多，可以实现，比如tianshou,rllib，用来操作替换算法部分，总之基本的内容作为我们操作学习的部分
- 新出的论文里面 sb3 里面很多没有实现，但是可以按照基本框架来进行操作实现

需要说明的是，stable_baselines3的侧重对象应该是初步接触深度强化的萌新玩家和偏强化学习理论研究的同学们，一旦遇到实际工程场景的研究和部署，我个人觉得还是自己搭建网络完成各种utils的搭建比较靠谱，stable_baselines3能够提供的utils不是细粒度的。

# 自定义环境实现